In [1]:
import torch

g:\Code\Python\GitHub\Portfolio-Optimizer\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
test = torch.randn((1, 8, 1))
x = (torch.tanh(test.squeeze()[3:])*0.9)
y = (torch.tanh(test.squeeze()[:3])*0.1)
z = torch.concat([y, x])

In [10]:
torch.softmax(z, dim=0)

tensor([0.1486, 0.1433, 0.1261, 0.0557, 0.1007, 0.0744, 0.2916, 0.0596])

In [48]:
import os
model_path = 'agent/saved_model/'
os.path.join(model_path, 'actor.ckpt')

'agent/saved_model/actor.ckpt'

In [16]:
import gym

In [17]:
env = gym.make('Pendulum-v1')